<a href="https://colab.research.google.com/github/NikoleSpace/TP-Compiladores/blob/main/Trabajo_Parcial_Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Instalación de los paquetes:
* ANTLR
* JAVA

In [ ]:
#Actualización de los paquetes
!sudo apt-get update
!sudo apt-get upgrade

In [ ]:
#Instalación de openJDK 17 para java
!apt-get install -y openjdk-17-jdk

In [ ]:
#Ver la versión actual de java
!java --version

In [ ]:
#Se actualizan las variables de entorno
!update-alternatives --config java

In [ ]:
#Instalando antlr4 mediante python
!python -m pip install antlr4-tools

In [ ]:
#Instalando los paquetes de antlr4
!antlr4

In [ ]:
#!git clone https://github.com/lmcanavals/compilers.git

In [ ]:
#Comprobando si esta el archivo jar de antlr4
!cat ~/.m2/repository/org/antlr/antlr4/4.13.1/antlr4-4.13.1-complete.jar
#El archivo es gigantesco!

##2. Creación de los analizadores sintatico, lexico

In [ ]:
%%writefile DOT.g4
grammar DOT;

graph
   : STRICT? ( GRAPH | DIGRAPH ) id_? '{' stmt_list '}' EOF
   ;

stmt_list
   : ( stmt ';'? )*
   ;

stmt
   : node_stmt | edge_stmt | attr_stmt | id_ '=' id_ | subgraph
   ;

attr_stmt
   : ( GRAPH | NODE | EDGE ) attr_list
   ;

attr_list
   : ( '[' a_list? ']' )+
   ;

a_list
   : ( id_ ( '=' id_ )? ','? )+
   ;

edge_stmt
   : ( node_id | subgraph ) edgeRHS attr_list?
   ;

edgeRHS
   : ( edgeop ( node_id | subgraph ) )+
   ;


edgeop
   : '->' | '--'
   ;


node_stmt
   : node_id attr_list?
   ;

node_id
   : id_ port?
   ;

port
   : ':' id_ ( ':' id_ )?
   ;

subgraph
   : ( SUBGRAPH id_? )? '{' stmt_list '}'
   ;

id_
   : ID | STRING | HTML_STRING | NUMBER
   ;

compass_pt
   : 'n' | 'ne' | 'e' | 'se' | 's' | 'sw' | 'w' | 'nw' | 'c' | '_';

STRICT
   : [Ss] [Tt] [Rr] [Ii] [Cc] [Tt]
   ;


GRAPH
   : [Gg] [Rr] [Aa] [Pp] [Hh]
   ;


DIGRAPH
   : [Dd] [Ii] [Gg] [Rr] [Aa] [Pp] [Hh]
   ;


NODE
   : [Nn] [Oo] [Dd] [Ee]
   ;


EDGE
   : [Ee] [Dd] [Gg] [Ee]
   ;


SUBGRAPH
   : [Ss] [Uu] [Bb] [Gg] [Rr] [Aa] [Pp] [Hh]
   ;


NUMBER
   : '-'? ( '.' DIGIT+ | DIGIT+ ( '.' DIGIT* )? )
   ;


fragment DIGIT
   : [0-9]
   ;


STRING
   : '"' ( '\\"' | . )*? '"'
   ;

ID
   : LETTER ( LETTER | DIGIT )*
   ;


fragment LETTER
   : [a-zA-Z\u0080-\u00FF_]
   ;


HTML_STRING
   : '<' ( TAG | ~ [<>] )* '>'
   ;


fragment TAG
   : '<' .*? '>'
   ;


COMMENT
   : '/*' .*? '*/' -> skip
   ;


LINE_COMMENT
   : '//' .*? '\r'? '\n' -> skip
   ;


PREPROC
   : '#' ~[\r\n]* -> skip
   ;


WS
   : [ \t\n\r]+ -> skip
   ;

Writing DOT.g4


In [41]:
#Se generan los arhivos de lexer, parser y visitor(semantico) de nuestra grámatica
!antlr4 DOT.g4
!ls DOT*.java

DOTBaseListener.java	DOTLexer.java	  DOTParser.java
DOTCustomListener.java	DOTListener.java


In [42]:
!javac -cp antlr-4.13.1-complete.jar DOT*.java

In [43]:
#Implementando la tabal de símbolos
%%writefile DOTCustomListener.java
import java.util.HashMap;
import java.util.Map;

public class DOTCustomListener extends DOTBaseListener {
    private Map<String, String> symbolTable = new HashMap<>();

    @Override
    public void exitNode_stmt(DOTParser.Node_stmtContext ctx) {
        String nodeId = ctx.node_id().id_().getText();
        //Añadir el nodo a la tabla de símbolos
        symbolTable.put(nodeId, "Node");
    }

    @Override
    public void exitEdge_stmt(DOTParser.Edge_stmtContext ctx) {

    }


    public Map<String, String> getSymbolTable() {
        return symbolTable;
    }
}


Overwriting DOTCustomListener.java


In [44]:
#Usando la función Main para usar la tabla de símbolos
%%writefile Main.java
import org.antlr.v4.runtime.CharStreams;
import org.antlr.v4.runtime.CommonTokenStream;

public class Main {
    public static void main(String[] args) {
        String input = "DOTCustomListener";  //La entrada DOTCustomListener

        DOTLexer lexer = new DOTLexer(CharStreams.fromString(input));
        DOTParser parser = new DOTParser(new CommonTokenStream(lexer));

        DOTCustomListener listener = new DOTCustomListener();
        parser.addParseListener(listener);

        //Parsear el input mediante la primera regla
        parser.graph();

        //Obtener la tabla de símbolos
        Map<String, String> symbolTable = listener.getSymbolTable();
    }
}


Overwriting Main.java


In [45]:
!javac -cp antlr-4.13.1-complete.jar Main.java

Main.java:8: error: cannot find symbol
        DOTLexer lexer = new DOTLexer(CharStreams.fromString(input));
        ^
  symbol:   class DOTLexer
  location: class Main
Main.java:8: error: cannot find symbol
        DOTLexer lexer = new DOTLexer(CharStreams.fromString(input));
                             ^
  symbol:   class DOTLexer
  location: class Main
Main.java:9: error: cannot find symbol
        DOTParser parser = new DOTParser(new CommonTokenStream(lexer));
        ^
  symbol:   class DOTParser
  location: class Main
Main.java:9: error: cannot find symbol
        DOTParser parser = new DOTParser(new CommonTokenStream(lexer));
                               ^
  symbol:   class DOTParser
  location: class Main
Main.java:11: error: cannot find symbol
        DOTCustomListener listener = new DOTCustomListener();
        ^
  symbol:   class DOTCustomListener
  location: class Main
Main.java:11: error: cannot find symbol
        DOTCustomListener listener = new DOTCustomListener();
 

# 3. Traducción de árbol sintáctico generado

In [ ]:
%%writefile ejemplo.expr
digraph G {
  A [label="Nodo A", shape=ellipse];
  B [label="Nodo B", shape=box];
  C [label="Nodo C", shape=diamond];

  A -> B [label="A -> B", color=blue];
  B -> C [label="B -> C", color=red];
  C -> A [label="C -> A", color=green];
}


Writing ejemplo.expr


In [ ]:
#Compilando nuestra grámatica usando el parser de antlr4, este nos devolverá el arbol sintactico
!antlr4-parse DOT.g4 graph -tree ejemplo.expr

(graph:2 digraph (id_:1 G) { (stmt_list:1 (stmt:1 (node_stmt:1 (node_id:1 (id_:1 A)) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "Nodo A") , (id_:1 shape) = (id_:1 ellipse)) ]))) ; (stmt:1 (node_stmt:1 (node_id:1 (id_:1 B)) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "Nodo B") , (id_:1 shape) = (id_:1 box)) ]))) ; (stmt:1 (node_stmt:1 (node_id:1 (id_:1 C)) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "Nodo C") , (id_:1 shape) = (id_:1 diamond)) ]))) ; (stmt:2 (edge_stmt:1 (node_id:1 (id_:1 A)) (edgeRHS:1 (edgeop:1 ->) (node_id:1 (id_:1 B))) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "A -> B") , (id_:1 color) = (id_:1 blue)) ]))) ; (stmt:2 (edge_stmt:1 (node_id:1 (id_:1 B)) (edgeRHS:1 (edgeop:1 ->) (node_id:1 (id_:1 C))) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "B -> C") , (id_:1 color) = (id_:1 red)) ]))) ; (stmt:2 (edge_stmt:1 (node_id:1 (id_:1 C)) (edgeRHS:1 (edgeop:1 ->) (node_id:1 (id_:1 A))) (attr_list:1 [ (a_list:1 (id_:1 label) = (id_:1 "C -> A") , (id_:1 

In [ ]:
#Se guarda el arbol sintactico en un archivo de texto plano -> arbol.txt
!antlr4-parse DOT.g4 graph -tree ejemplo.expr > arbol.txt

In [ ]:
import re
import numpy as np
import csv

def parse_input_data(input_data):
    edges = []
    nodes = set()

    # Usando regex para extraer la información raíz
    edge_pattern = r'\(node_id:\d+ \(id_:1 (\w+)\)\)\s*\(edgeRHS:\d+ \(edgeop:1 ->\) \(node_id:\d+ \(id_:1 (\w+)\)\)\)'

    matches = re.findall(edge_pattern, input_data)
    for source, target in matches:
        edges.append((source, target))
        nodes.add(source)
        nodes.add(target)

    nodes = sorted(list(nodes))
    node_to_index = {node: index for index, node in enumerate(nodes)}

    # Creando la matriz de adyacencia
    num_nodes = len(nodes)
    adjacency_matrix = np.zeros((num_nodes, num_nodes))

    for source, target in edges:
        source_index = node_to_index[source]
        target_index = node_to_index[target]
        adjacency_matrix[source_index][target_index] = 1

    return adjacency_matrix, nodes

def save_adjacency_matrix_to_csv(adjacency_matrix, nodes, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([''] + nodes)  # La rama Header
        for node, row in zip(nodes, adjacency_matrix):
            writer.writerow([node] + row.tolist())

#Datos de entrada
with open('arbol.txt', 'r') as file:
    input_data = file.read()

# Parsear los datos y generar la matriz de adyacencia
adjacency_matrix, nodes = parse_input_data(input_data)
print(adjacency_matrix)
# Guardar la matriz de adyacencia en un archivo CSV
save_adjacency_matrix_to_csv(adjacency_matrix, nodes, 'adjacency_matrix.csv')


[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [38]:
#Instalación interna de ANTLR4
!curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar
#No se puede hacer el CLASSPATH para tener dentro de ahí nuestro antlr. Esto por el funcionamiento de las celdas

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2089k  100 2089k    0     0  3917k      0 --:--:-- --:--:-- --:--:-- 3919k


In [ ]:
#Ejecución de nuestro antlr4 de forma manual usando el grun
grun('DOT graph -tree ejemplo.expr')

In [ ]:
!java -Xmx500M -cp "/content/antlr-4.13.1-complete.jar:$CLASSPATH" org.antlr.v4.gui.TestRig DOT graph -tree ejemplo.expr

(graph digraph (id_ G) { (stmt_list (stmt (node_stmt (node_id (id_ A)) (attr_list [ (a_list (id_ label) = (id_ "Nodo A") , (id_ shape) = (id_ ellipse)) ]))) ; (stmt (node_stmt (node_id (id_ B)) (attr_list [ (a_list (id_ label) = (id_ "Nodo B") , (id_ shape) = (id_ box)) ]))) ; (stmt (node_stmt (node_id (id_ C)) (attr_list [ (a_list (id_ label) = (id_ "Nodo C") , (id_ shape) = (id_ diamond)) ]))) ; (stmt (edge_stmt (node_id (id_ A)) (edgeRHS (edgeop ->) (node_id (id_ B))) (attr_list [ (a_list (id_ label) = (id_ "A -> B") , (id_ color) = (id_ blue)) ]))) ; (stmt (edge_stmt (node_id (id_ B)) (edgeRHS (edgeop ->) (node_id (id_ C))) (attr_list [ (a_list (id_ label) = (id_ "B -> C") , (id_ color) = (id_ red)) ]))) ; (stmt (edge_stmt (node_id (id_ C)) (edgeRHS (edgeop ->) (node_id (id_ A))) (attr_list [ (a_list (id_ label) = (id_ "C -> A") , (id_ color) = (id_ green)) ]))) ;) } <EOF>)


In [ ]:

##Comprobar nuestro antlr4-tools en java
!java org.antlr.v4.Tool